In [2]:
#导入数据
import pandas as pd
content = pd.read_csv('./sqlResult_1558435.csv', encoding='gb18030')
data = pd.DataFrame({'content':content['content'],'y':content['source']=='新华社'})
data=data.dropna(axis=0)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import cut_sentence as cs
from collections import defaultdict

#1. 统计词频
tf=defaultdict(int)
for index,row in data.iterrows():
    words_list = cs.segment(row['content'],type="arr")
    for word in words_list:
        tf[word]+=1

result = sorted(tf.items(),key=lambda item: item[1])
#3. 挑选词频大于150小于350的词作为feature
# 构建vocabulary
vocabulary = defaultdict(int)
index = 0
for item in result:
    if item[1]>150 and item[1]<350:
        vocabulary[item[0]]=index
        index+=1
print(len(vocabulary))

from scipy.sparse import dok_matrix

def flatten2line(doc):
    #把一篇文档压成一行
    raw_doc=""
    for sentence in doc:
        raw_doc+=sentence
        raw_doc+=" "
#     print(raw_doc)
    return raw_doc

docs=[]
for index,row in data.iterrows():
#     if index>1: break
    doc = cs.cut_and_segment(row['content'])
    if len(doc)==0:
#         print(index,row['content'],doc)
        docs.append("")
        continue
#    print(doc)
    doc = flatten2line(doc)
    docs.append(doc)

tfidf = TfidfVectorizer(max_features=len(vocabulary),vocabulary=vocabulary)
vectorize = tfidf.fit_transform(docs)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectorize,data['y'])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Raytine\AppData\Local\Temp\jieba.cache
Loading model cost 0.581 seconds.
Prefix dict has been built succesfully.


5294


In [22]:
def search_super_params(parameters,estimator):
    scores = ['f1']
    for score in scores:
        print('%s_weighted: ' % score)
        est = estimator()
        clf = GridSearchCV(est,parameters,cv=3,scoring='%s_weighted' % score,verbose=3)
        clf.fit(X_train.todense(),y_train)
        print(clf.best_params_)
        y_true, y_pred = y_test, clf.predict(X_test.todense())
        print(classification_report(y_true, y_pred))

### 支持向量机

In [7]:
# SVM
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0)
clf.fit(X_train, y_train)
print(X_train.shape)

# print(y_train)
y_hat = clf.predict(X_test)

from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

print(precision_score(y_test,y_hat))
print(recall_score(y_test,y_hat))
print(f1_score(y_test,y_hat))

(65290, 5294)
0.9548990645002462
0.9840673838035315
0.9692638312759257


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# scores = ['f1']
parameters={'tol':[1e-4,1e-3,1e-2]}
search_super_params(parameters,lambda : LinearSVC())
# for score in scores:
#     print('%s_weighted: ' % score)
#     svc = LinearSVC(random_state=0)
#     clf = GridSearchCV(svc,parameters,cv=3,scoring='%s_weighted' % score,verbose=3)
#     clf.fit(X_train,y_train)
#     print(clf.best_params_)
#     y_true, y_pred = y_test, clf.predict(X_test)
#     print(classification_report(y_true, y_pred))

f1_weighted: 
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] tol=0.0001 ......................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............. tol=0.0001, score=0.9349310926139736, total=   2.2s
[CV] tol=0.0001 ......................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.7s remaining:    0.0s


[CV] ............. tol=0.0001, score=0.9356977941321748, total=   1.6s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.9s remaining:    0.0s


[CV] tol=0.0001 ......................................................
[CV] ............. tol=0.0001, score=0.9354533857011122, total=   1.6s
[CV] tol=0.001 .......................................................
[CV] .............. tol=0.001, score=0.9349310926139736, total=   1.9s
[CV] tol=0.001 .......................................................
[CV] .............. tol=0.001, score=0.9356977941321748, total=   2.0s
[CV] tol=0.001 .......................................................
[CV] .............. tol=0.001, score=0.9354938317788455, total=   1.6s
[CV] tol=0.01 ........................................................
[CV] ............... tol=0.01, score=0.9350344418184231, total=   1.6s
[CV] tol=0.01 ........................................................
[CV] ............... tol=0.01, score=0.9356977941321748, total=   1.6s
[CV] tol=0.01 ........................................................
[CV] ............... tol=0.01, score=0.9355519301504234, total=   1.6s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   21.2s finished


{'tol': 0.01}
              precision    recall  f1-score   support

       False       0.78      0.55      0.65      2056
        True       0.95      0.98      0.97     19708

   micro avg       0.94      0.94      0.94     21764
   macro avg       0.87      0.77      0.81     21764
weighted avg       0.94      0.94      0.94     21764



### Bayes

In [24]:
from sklearn.naive_bayes import GaussianNB

parameters={}
search_super_params(parameters,lambda : GaussianNB())

f1_weighted: 
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....................... , score=0.7137695468965951, total=   8.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.6s remaining:    0.0s


[CV] ....................... , score=0.7241695895010841, total=   7.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   26.3s remaining:    0.0s


[CV] ....................... , score=0.7211975777443014, total=   7.4s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   38.9s finished


{}
              precision    recall  f1-score   support

       False       0.20      0.92      0.33      2056
        True       0.99      0.62      0.76     19708

   micro avg       0.65      0.65      0.65     21764
   macro avg       0.59      0.77      0.55     21764
weighted avg       0.91      0.65      0.72     21764



### Decision Tree

In [25]:

from sklearn.tree import DecisionTreeClassifier

parameters={'max_depth':[5,10,None]}
search_super_params(parameters,lambda : DecisionTreeClassifier())

f1_weighted: 
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] max_depth=5 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ max_depth=5, score=0.8828361963584322, total=  21.0s
[CV] max_depth=5 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.7s remaining:    0.0s


[CV] ............. max_depth=5, score=0.885352087456465, total=  20.7s
[CV] max_depth=5 .....................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   43.4s remaining:    0.0s


[CV] ............ max_depth=5, score=0.8867481020771206, total=  20.7s
[CV] max_depth=10 ....................................................
[CV] ........... max_depth=10, score=0.8982670788965637, total=  39.3s
[CV] max_depth=10 ....................................................
[CV] ........... max_depth=10, score=0.9000815523608732, total=  39.4s
[CV] max_depth=10 ....................................................
[CV] ........... max_depth=10, score=0.9030973823263906, total=  39.4s
[CV] max_depth=None ..................................................
[CV] ......... max_depth=None, score=0.9229082157726919, total=55.2min
[CV] max_depth=None ..................................................
[CV] ......... max_depth=None, score=0.9218968048031095, total=55.0min
[CV] max_depth=None ..................................................
[CV] ......... max_depth=None, score=0.9259087788409612, total=46.8min


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 160.1min finished


{'max_depth': None}
              precision    recall  f1-score   support

       False       0.69      0.56      0.62      2056
        True       0.95      0.97      0.96     19708

   micro avg       0.93      0.93      0.93     21764
   macro avg       0.82      0.77      0.79     21764
weighted avg       0.93      0.93      0.93     21764



### Random Forest

In [27]:
from sklearn.ensemble import RandomForestClassifier

parameters={'max_depth':[5,10,15,None]}
search_super_params(parameters,lambda : RandomForestClassifier())

f1_weighted: 
Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] max_depth=5 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
E:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV] ............ max_depth=5, score=0.8608445547989465, total=   3.5s
[CV] max_depth=5 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.4s remaining:    0.0s
E:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV] ............ max_depth=5, score=0.8612331591387459, total=   3.1s
[CV] max_depth=5 .....................................................


...

[CV] ......... max_depth=None, score=0.9331985921316458, total= 4.2min
[CV] max_depth=None ..................................................


E:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[CV] ......... max_depth=None, score=0.9313887267745607, total= 4.1min


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 13.4min finished
E:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


{'max_depth': None}
              precision    recall  f1-score   support

       False       0.80      0.53      0.64      2056
        True       0.95      0.99      0.97     19708

   micro avg       0.94      0.94      0.94     21764
   macro avg       0.87      0.76      0.80     21764
weighted avg       0.94      0.94      0.94     21764

